In [1]:
!pip install -q transformers

In [2]:
from transformers import AutoProcessor, Wav2Vec2Model
import torch
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import scale
from torchsummary import summary
from matplotlib.font_manager import FontProperties

torch.__version__


'2.1.0+cu118'

In [3]:
#加载模型
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# 加载音频文件
audio_input, rate = librosa.load("test.wav", sr = 16000)

inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)
list(last_hidden_states.shape)


tensor([[[ 0.0184,  0.0249, -0.0534,  ..., -0.1300,  0.0519,  0.1058],
         [-0.0644,  0.0021, -0.0993,  ..., -0.2256,  0.0159,  0.1310],
         [-0.0644,  0.0021, -0.0993,  ..., -0.2255,  0.0159,  0.1310],
         ...,
         [-0.0071,  0.0287, -0.0498,  ..., -0.1711,  0.0360,  0.1172],
         [-0.0186,  0.0125, -0.0654,  ..., -0.1694,  0.0417,  0.1313],
         [-0.0159,  0.0108, -0.0674,  ..., -0.1684,  0.0438,  0.1396]]])


[1, 171, 768]

In [5]:
jawopen = [
            0.009586737491190434,
            0.008979345671832561,
            0.011321083642542362,
            0.010439089499413967,
            0.009360973723232746,
            0.006232088897377253,
            0.004848846234381199,
            0.009360488504171371,
            0.013829204253852367,
            0.017454467713832855,
            0.0196081455796957,
            0.04048984497785568,
            0.07197241485118866,
            0.06341962516307831,
            0.052364006638526917,
            0.013461578637361526,
            0.011079512536525726,
            0.006322343833744526,
            0.004936319775879383,
            0.005036789923906326,
            0.004498274996876717,
            0.004021129570901394,
            0.0048559606075286865,
            0.005624715238809586,
            0.005461822263896465,
            0.006910244934260845,
            0.004959948360919952,
            0.007502279244363308,
            0.009119633585214615,
            0.013383540324866772,
            0.02590612694621086,
            0.01770544797182083,
            0.00786164402961731,
            0.010478793643414974,
            0.027493944391608238,
            0.08369234949350357,
            0.11772142350673676,
            0.0706266537308693,
            0.015011589042842388,
            0.006985705345869064,
            0.0074021099135279655,
            0.006355965044349432,
            0.00825462769716978,
            0.012373908422887325,
            0.009418923407793045,
            0.009582826867699623,
            0.008861105889081955,
            0.004849016200751066,
            0.010531198233366013,
            0.009303479455411434,
            0.008775768801569939,
            0.006546579767018557,
            0.0034988783299922943,
            0.0034243701957166195,
            0.004655267111957073,
            0.005182082299143076,
            0.005518872756510973,
            0.006342220585793257,
            0.009763234294950962,
            0.0068406276404857635,
            0.0052495053969323635,
            0.00584684731438756,
            0.010323402471840382,
            0.027268482372164726,
            0.03300449624657631,
            0.01794753596186638,
            0.010961898602545261,
            0.005985470954328775,
            0.00395447202026844,
            0.0030053944792598486,
            0.0024215253069996834,
            0.002878825878724456,
            0.002389328321442008,
            0.002357768826186657,
            0.002324940636754036,
            0.0021527810022234917,
            0.004213900770992041,
            0.0046121045015752316,
            0.005094058811664581,
            0.006326064933091402,
            0.005623629782348871,
            0.005182364955544472,
            0.005025811493396759
        ]

# 将 Python 列表转换为 NumPy 数组
my_array = np.array(jawopen)

# 现在，my_array 是一个 NumPy 数组
print(my_array)
print(my_array.shape)

[0.00958674 0.00897935 0.01132108 0.01043909 0.00936097 0.00623209
 0.00484885 0.00936049 0.0138292  0.01745447 0.01960815 0.04048984
 0.07197241 0.06341963 0.05236401 0.01346158 0.01107951 0.00632234
 0.00493632 0.00503679 0.00449827 0.00402113 0.00485596 0.00562472
 0.00546182 0.00691024 0.00495995 0.00750228 0.00911963 0.01338354
 0.02590613 0.01770545 0.00786164 0.01047879 0.02749394 0.08369235
 0.11772142 0.07062665 0.01501159 0.00698571 0.00740211 0.00635597
 0.00825463 0.01237391 0.00941892 0.00958283 0.00886111 0.00484902
 0.0105312  0.00930348 0.00877577 0.00654658 0.00349888 0.00342437
 0.00465527 0.00518208 0.00551887 0.00634222 0.00976323 0.00684063
 0.00524951 0.00584685 0.0103234  0.02726848 0.0330045  0.01794754
 0.0109619  0.00598547 0.00395447 0.00300539 0.00242153 0.00287883
 0.00238933 0.00235777 0.00232494 0.00215278 0.0042139  0.0046121
 0.00509406 0.00632606 0.00562363 0.00518236 0.00502581]
(83,)


In [6]:
last_hidden_states = torch.tensor(last_hidden_states)  # 将列表转换为张量
jawopen = torch.tensor(jawopen)  # 将列表转换为张量



# 使用切片进行降采样
downsampled_last_hidden_states = last_hidden_states[:, ::2, :]

downsampled_last_hidden_states.shape

# 删除最后三个元素
downsampled_last_hidden_states = downsampled_last_hidden_states[:, :-3, :]

# 删除最后三个元素
downsampled_last_hidden_states.shape



# downsampled_last_hidden_states 现在具有与 jawopen 相匹配的帧数

<ipython-input-6-a5b1b24fdf07>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_hidden_states = torch.tensor(last_hidden_states)  # 将列表转换为张量


torch.Size([1, 83, 768])

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# 重塑 x 和 y 的形状以适应线性回归
x_reshaped = downsampled_last_hidden_states.reshape(-1, 768)
y_reshaped = jawopen.reshape(-1, 1)

print(x_reshaped.shape)
print(x_reshaped)
print(y_reshaped.shape)
print(y_reshaped)

# 划分数据集为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y_reshaped, test_size=0.2, random_state=42)

# 创建线性回归模型
regression_model = LinearRegression()

# 训练线性回归模型
regression_model.fit(x_train, y_train)

# 进行预测
y_pred = regression_model.predict(x_test)


# 计算均方误差（MSE）作为回归任务的评估指标
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE) on the test set: {mse}")

torch.Size([83, 768])
tensor([[ 0.0184,  0.0249, -0.0534,  ..., -0.1300,  0.0519,  0.1058],
        [-0.0644,  0.0021, -0.0993,  ..., -0.2255,  0.0159,  0.1310],
        [-0.0644,  0.0021, -0.0993,  ..., -0.2256,  0.0158,  0.1310],
        ...,
        [-0.0644,  0.0021, -0.0993,  ..., -0.2255,  0.0159,  0.1310],
        [-0.0067,  0.0254, -0.0543,  ..., -0.1658,  0.0385,  0.1283],
        [-0.0012,  0.0379, -0.0554,  ..., -0.1630,  0.0375,  0.1130]])
torch.Size([83, 1])
tensor([[0.0096],
        [0.0090],
        [0.0113],
        [0.0104],
        [0.0094],
        [0.0062],
        [0.0048],
        [0.0094],
        [0.0138],
        [0.0175],
        [0.0196],
        [0.0405],
        [0.0720],
        [0.0634],
        [0.0524],
        [0.0135],
        [0.0111],
        [0.0063],
        [0.0049],
        [0.0050],
        [0.0045],
        [0.0040],
        [0.0049],
        [0.0056],
        [0.0055],
        [0.0069],
        [0.0050],
        [0.0075],
        [0.0091],
  

In [8]:
print(y_pred.shape)
print(y_pred)


(17, 1)
[[-0.63841295]
 [-0.4173298 ]
 [-0.64848566]
 [ 0.9359231 ]
 [ 0.01497281]
 [-0.02761388]
 [ 0.06403399]
 [-0.86188626]
 [-0.16345656]
 [ 0.36883712]
 [ 0.12831187]
 [ 0.8802986 ]
 [ 1.279058  ]
 [ 0.5686691 ]
 [-0.8637111 ]
 [ 0.51157784]
 [-0.05655253]]


In [9]:
from sklearn.metrics import r2_score

# 计算 R²
r_squared = r2_score(y_test, y_pred)

r_squared

-711.2346732617685

In [10]:

#reg.coef_ 参数w1,w2.....wn
print(regression_model.coef_)

#截距  reg.intercept_
print(regression_model.intercept_)

[[-2.51362413e-01  1.77734205e-03 -1.46696121e-02  5.19804005e-03
   1.15538388e-01 -2.09535196e-01  5.14661908e-01  2.66111717e-02
   6.86329305e-02 -9.05385911e-02  5.96994907e-02 -4.70045209e-01
   6.06446601e-02  5.55610433e-02  5.42309821e-01 -2.00378388e-01
   4.96739626e-01  5.49489446e-02 -1.64525211e-01 -4.60145250e-02
   1.76509507e-02  1.51052494e-02 -3.14922690e-01  2.43417099e-02
  -9.81973290e-01  1.55508369e-01  9.45163012e-01  7.30857626e-02
   1.59399778e-01 -9.87710282e-02 -2.12319165e-01 -8.16229172e-03
   2.41257101e-01 -5.78076579e-02  4.50368732e-01 -3.11720759e-01
   4.62169230e-01 -4.27367818e-03 -4.40182924e-01  3.75403970e-01
   4.31060493e-01  4.17255640e-01  2.49688864e-01  5.37195019e-02
   3.29122514e-01  3.89791042e-01 -6.82985783e-02  2.86792368e-01
  -3.49610895e-01  3.11231595e-02 -8.28459114e-03  6.68186098e-02
   1.34195000e-01 -6.89229190e-01 -1.37422420e-02  3.53674412e-01
   7.43700713e-02  3.25570643e-01 -1.04983553e-01 -1.29334778e-01
  -1.37699

In [ ]:
#支持向量机

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error



# 数据标准化
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# 创建支持向量机回归模型
svm_regressor = SVR(C=10000, kernel='rbf', gamma='scale')
svm_regressor.fit(x_train_scaled, y_train)

# 预测
y_pred_svm = svm_regressor.predict(x_test_scaled)

# 评估性能
mse_svm = mean_squared_error(y_test, y_pred_svm)
print(f"Mean Squared Error (SVR): {mse_svm}")


Mean Squared Error (SVR): 0.002339032649578258


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
print(y_pred_svm.shape)
print(y_pred_svm)

(17,)
[0.06002318 0.06002318 0.06002318 0.06002318 0.06002318 0.06002318
 0.06002318 0.06002318 0.06002318 0.06002318 0.06002318 0.06002318
 0.06002318 0.06002318 0.06002318 0.06002318 0.06002318]


In [ ]:
#神经网络


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np


X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(x_test, dtype=torch.float32)

# 定义神经网络模型
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化神经网络模型
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NeuralNetwork(input_size, hidden_size, output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 200
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 打印训练信息
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 模型预测
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# 转换为NumPy数组
y_pred = y_pred_tensor.numpy()

# 评估性能
mse_nn = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (Neural Network): {mse_nn}")


<ipython-input-16-bb720996fc16>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
<ipython-input-16-bb720996fc16>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-16-bb720996fc16>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(x_test, dtype=torch.float32)


Epoch [100/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000
Mean Squared Error (Neural Network): 0.00132011401001364


In [17]:
print(y_pred)
print(y_test)

[[ 0.00215594]
 [ 0.01098787]
 [-0.02917218]
 [-0.02131323]
 [-0.02054003]
 [ 0.01233904]
 [ 0.0462926 ]
 [ 0.03147978]
 [ 0.00760338]
 [-0.01706246]
 [ 0.00213167]
 [ 0.0525396 ]
 [-0.04245719]
 [ 0.01727333]
 [ 0.00263963]
 [ 0.03335863]
 [ 0.01772334]]
tensor([[0.0259],
        [0.0096],
        [0.0049],
        [0.0177],
        [0.0049],
        [0.0091],
        [0.0196],
        [0.0034],
        [0.0094],
        [0.0720],
        [0.0093],
        [0.0105],
        [0.0030],
        [0.0837],
        [0.0024],
        [0.0096],
        [0.0022]])


In [18]:
from sklearn.metrics import r2_score

# 计算 R²
r_squared = r2_score(y_test, y_pred)

r_squared

-1.4960725450959727